In [ ]:
%matplotlib inline

import seaborn as sns
import pandas as pd
from pysal.lib import weights
from pysal.explore import esda
from splot.esda import moran_scatterplot, lisa_cluster, plot_local_autocorrelation
import geopandas as gpd
import numpy as np
import contextily as ctx
import matplotlib.pyplot as plt

np.random.seed(123)

In [ ]:
# file path
br_path = 'C:\\Users\\MSI\\Documents\\Data_one\\brexit.gpkg'

In [ ]:
# Load local Authority geometries using their ID code as index
br = gpd.read_file(br_path).set_index('LAD14CD', drop=False)
ax = br.plot(figsize=(9, 9), alpha=0.5, color='red')
# Add background map, expressing target CRS so the basemap can be
# reprojected (warped)
ctx.add_basemap(ax, crs=br.crs)

In [ ]:
# Set up figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot % to leave
br.plot(column='Pct_Leave', scheme='Quantiles', 
        legend=True, ax=ax)
# Remove axis frame
ax.set_axis_off()
# Change background color of the figure
f.set_facecolor('0.75')
# Title
f.suptitle('% to Leave', size=30)
# Draw
plt.show()

In [ ]:
#Create the spatial weights matrix
%time w = weights.Queen.from_dataframe(br, idVariable='LAD14CD')

In [ ]:
# check the neighborhood of the observation
w['E08000012']

In [ ]:
# inspect and map the islands
ax = br.plot(color='k', figsize=(9, 9))
br.loc[w.islands, :].plot(color='red', ax=ax);


In [ ]:
# drop the ones that ........
br = br.drop(w.islands)

In [ ]:
# recalculate weight matrix for .....
%time w = weights.Queen.from_dataframe(br, idVariable='LAD14CD')


In [ ]:
# Row standardize the matrix
w.transform = 'R'

In [ ]:
# check the neighbors of the observation
w['E08000012']

In [ ]:
# calculate the spatial lag for the variable `Pct_Leave` and store it directly in the main table
br['w_Pct_Leave'] = weights.lag_spatial(w, br['Pct_Leave'])

In [ ]:
# look at the resulting variable
br[['LAD14NM', 'Pct_Leave', 'w_Pct_Leave']].head()

In [ ]:
#fact check this is correct by querying the spatial weights matrix to find out Hartepool's neighbors
w.neighbors['E06000001']

In [ ]:
# checking their values
neis = br.loc[w.neighbors['E06000001'], 'Pct_Leave']
neis

In [ ]:
#average value
neis.mean()

In [ ]:
#Standardizing means to substract the average value and divide by the standard deviation each observation of the column
br['Pct_Leave_std'] = (br['Pct_Leave'] - br['Pct_Leave'].mean()) / br['Pct_Leave'].std()

In [ ]:
#recreate its spatial lag
br['w_Pct_Leave_std'] = weights.lag_spatial(w, br['Pct_Leave_std'])

In [ ]:
# Moran Plot

# Setup the figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot values
sns.regplot(x='Pct_Leave_std', y='w_Pct_Leave_std', data=br, ci=None)
# Add vertical and horizontal lines
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
# Display
plt.show()

In [ ]:
# calculate Moran's I
mi = esda.Moran(br['Pct_Leave'], w)

In [ ]:
# retrieve the Moran's I statistic value 
mi.I

In [ ]:
#  p-value for Moran's I (the lesser that better, like 0.1% is considered statistically significant)
mi.p_sim

In [ ]:
#
moran_scatterplot(mi);

In [ ]:
# Spatial Auto coreelation

# Local Indicators of Spatial Association

# Setup the figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot values
sns.regplot(x='Pct_Leave_std', y='w_Pct_Leave_std', data=br, ci=None)
# Add vertical and horizontal lines
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
plt.text(1.75, 0.5, "HH", fontsize=25)
plt.text(1.5, -1.5, "HL", fontsize=25)
plt.text(-2, 1, "LH", fontsize=25)
plt.text(-1.5, -2.5, "LL", fontsize=25)
# Display
plt.show()

In [ ]:
#
lisa = esda.Moran_Local(br['Pct_Leave'], w)

In [ ]:
#
# Break observations into significant or not
br['significant'] = lisa.p_sim < 0.05
# Store the quadrant they belong to
br['quadrant'] = lisa.q


In [ ]:
#
br['significant'].head()

In [ ]:
#
lisa.p_sim[:5]

In [ ]:
#
br['quadrant'].head()

In [ ]:
# LISA cluster map
lisa_cluster(lisa, br);

In [ ]:
# Customized LISA map

# Setup the figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))
# Plot insignificant clusters
ns = br.loc[br['significant']==False, 'geometry']

                         #edgecolor = "", linewidth =    added
ns.plot(ax=ax, color='k', edgecolor = "white", linewidth = 0.5)
# Plot HH clusters
hh = br.loc[(br['quadrant']==1) & (br['significant']==True), 'geometry']

hh.plot(ax=ax, color='red', edgecolor = "white", linewidth = 0.5)

# Plot LL clusters
ll = br.loc[(br['quadrant']==3) & (br['significant']==True), 'geometry']

ll.plot(ax=ax, color='blue', edgecolor = "white", linewidth = 0.5)

# Plot LH clusters
lh = br.loc[(br['quadrant']==2) & (br['significant']==True), 'geometry']

lh.plot(ax=ax, color='#83cef4', edgecolor = "white", linewidth = 0.5)

# Plot HL clusters
hl = br.loc[(br['quadrant']==4) & (br['significant']==True), 'geometry']

hl.plot(ax=ax, color='#e59696', edgecolor = "white", linewidth = 0.5)

# Style and draw
f.suptitle('LISA for Brexit vote', size=30)
f.set_facecolor('0.75')

ax.set_axis_off()

plt.show()

In [ ]:
# display the outcome maps together

plot_local_autocorrelation(lisa, br, 'Pct_Leave');


In [ ]:
# line graph to show Moran's I changes over time
df = pd.read_csv('aaaaaaaaaaaaaaaa.csv')

plt.figure(figsize=(8, 5))

plt.title(' ? ', fontdict={'fontweight':'bold', 'fontsize': 18 })

plt.plot(df.Year, gas.USA, 'b.-', label='?')
plt.plot(df.Year, gas.Canada, 'r.-')
            # ticks are every 3 years in a row + adds 2011 at the end of the axis
plt.xticks(gas.Year[::3].tolist()+[2011])

plt.xlabel('Year')
plt.ylabel('US Dollars')

plt.legend()

plt.savefig('Gas_price_figure.png', dpi=300)

plt.show()

In [ ]:
# example of how to display the outcome with different shades of blue ( Chat Gpt )
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the GeoDataFrame for the provinces (replace with your file path)
gdf = gpd.read_file('path_to_your_provinces_shapefile.shp')

# Sample data for energy consumption and income (replace with your real data)
# Assume `data` contains 'province', 'energy_consumption', and 'income_level' columns
df = pd.read_csv('file path')

# Merge GeoDataFrame with data based on the province name
merged_gdf = gdf.merge(df, left_on='province_column_in_shapefile', right_on='province')

# Calculate the correlation between energy consumption and income level
merged_gdf['correlation'] = merged_gdf.apply(lambda row: row['energy_consumption'] / row['income_level'], axis=1)

# Set up the plot
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Plot the correlation as different shades of blue
# Create a continuous colormap for correlation values
cmap = sns.color_palette("Blues", as_cmap=True)

# Plot the provinces, coloring by the 'correlation' column
merged_gdf.plot(column='correlation', cmap=cmap, legend=True, ax=ax)

# Add mean income level as text in each province
for idx, row in merged_gdf.iterrows():
    plt.text(row.geometry.centroid.x, row.geometry.centroid.y, 
             s=f"{row['income_level']:.0f} €", 
             horizontalalignment='center', fontsize=8, color='black')

# Customize the plot
ax.set_title("Correlation between Energy Consumption and Income Level by Province", fontsize=16)
ax.set_axis_off()  # Hide axis

# Show the plot
plt.show()


In [ ]:
# to display Significance map ( chat gpt )

import geopandas as gpd  # Import geopandas for spatial data handling
import matplotlib.pyplot as plt  # Import matplotlib for plotting

# Load spatial data (shapefile) of the area
gdf = gpd.read_file('path_to_shapefile.shp')

# Add or load significance values into the GeoDataFrame
gdf['significance'] = [0.05, 0.01, 0.03, 0.07]  # Example significance values

# Set up a blue color map
cmap = 'Blues'

# Plot the map with different shades of blue based on significance values
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
gdf.plot(column='significance', cmap=cmap, legend=True, ax=ax)

# Add a title
plt.title("Spatial Distribution of Significance Values (Shades of Blue)")

# Show the plot
plt.show()

In [ ]:
# anothr way to display Significance map ( chat gpt ) 

# Load spatial data (shapefile) of the area
gdf = gpd.read_file('path_to_shapefile.shp')

# Add or load significance values into the GeoDataFrame
gdf['significance'] = [0.05, 0.01, 0.03, 0.07]  # Example significance values

# Set up a color map for significance values
cmap = sns.color_palette("coolwarm", as_cmap=True)

# Plot the map with significance values
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
gdf.plot(column='significance', cmap=cmap, legend=True, ax=ax)

# Add a title
plt.title("Spatial Distribution of Significance Values")

# Show the plot
plt.show()